In [1]:
import os 

In [2]:
%pwd

'/Users/arihantsingla/Documents/CDD_Project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/arihantsingla/Documents/CDD_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL : str
    local_data_file: Path
    unzip_dir: Path



In [6]:
from src.chicken_disease_classification.constants import *
from src.chicken_disease_classification.utils.common import read_yaml, create_directories

In [7]:
class configManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
    

In [8]:
import os 
import zipfile
import urllib.request as request
from src.chicken_disease_classification.utils.common import get_size
from src.chicken_disease_classification import logger

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                filename, headers = request.urlretrieve(url=self.config.source_URL, filename=self.config.local_data_file)
                logger.info(f"{filename} downloaded! with the following info: {headers}")
            except Exception as e:
                logger.error(f"Error occurred while downloading the file: {e}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

## path to where the unzipped file will be stored
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            


In [10]:
config = configManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()
try:
    print("Data ingestion completed successfully")  
except Exception as e:
    logger.error(f"An error occurred during data ingestion completion message: {e}")

[2025-03-20 16:39:46,085: INFO: common: Reading YAML file from: config/config.yaml]
[2025-03-20 16:39:46,087: INFO: common: YAML file read successfully.]
[2025-03-20 16:39:46,087: INFO: common: Reading YAML file from: params.yaml]
[2025-03-20 16:39:46,088: INFO: common: YAML file read successfully.]
[2025-03-20 16:39:46,088: INFO: common: created directory at: artifacts]
[2025-03-20 16:39:46,088: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-20 16:39:47,009: INFO: 2609200933: artifacts/data_ingestion/data.zip downloaded! with the following info: Server: GitHub.com
Date: Thu, 20 Mar 2025 11:09:46 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"082424049fc76aebfc7c0546595bfb8e"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Pro

BadZipFile: File is not a zip file